# YOLO Step 3 v.10
*withOUT Synthetic data*

Roboflow data generated:
* 2022-03-14 no_augmentation
* Version 12




In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import io
import pandas as pd
import json
import os
import glob 


In [6]:
!ls ../../

AutoArki  data	yolov5


In [11]:
!pip install roboflow
#os.chdir("../../yolov5")

from roboflow import Roboflow
rf = Roboflow(api_key="pvpFbwxA6FFS6pAwYdNk")
project = rf.workspace("new-workspace-s32a1").project("autoarki")
dataset = project.version(15).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
loading Roboflow workspace...
loading Roboflow project...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Extracting Dataset Version Zip to autoarki-15 in yolov5pytorch:: 100%|█| 5198/5198 [00:00<00:00, 11978.


Manually rename autoarki folder or update the directory path below

In [18]:
os.chdir("../yolov5")

In [21]:
# # change directory and copy files locally -- only do once
# # Roboflow output
#os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5")

!rm -R train; mkdir train
!rm -R test; mkdir test
!rm -R valid; mkdir valid
!cp /workspace/yolo/yolo/data/train/images/*  train --recursive
!cp /workspace/yolo/yolo/data/train/labels/*  train --recursive
!cp /workspace/yolo/yolo/data/test/images/*  test --recursive
!cp /workspace/yolo/yolo/data/test/labels/*  test --recursive
!cp /workspace/yolo/yolo/data/valid/images/*  valid --recursive
!cp /workspace/yolo/yolo/data/valid/labels/*  valid --recursive

# Check 

In [22]:
def count_classes(file_path):
    '''
    increment class_counts dictionary for each instance of a 
    class check against roboflow summary (health check)
    
    '''
    infile = open(file_path, "r")
    lines = infile.readlines()

    if len(lines) == 0:
        pass
    else:
        for og_line in lines:
            class_counts[og_line[0]] +=1
    return class_counts

names: ['Column_Wall_Intersection', 'cmu_hinge_clearance', 'floating_door', 'illegibletext', 'missing_wall', 'no_stair_direction']


**Class Order**  <br>
0: '-' <br>
1: Column_Wall_Intersection <br>
2: cmu_hinge_clearance <br>
3: floating_door <br>
4: illegibletext <br>
5: missing_wall <br>
6: n <br>
7: no_stair_direction <br>

In [24]:
#Count of class types in train
class_counts = {'0':0, '1':0, '2':0, '3': 0, '4': 0, '5': 0, '6':0, '7':0}

os.chdir("train")
for file in glob.glob("*.txt"):
    count_classes(file)
print(class_counts)

{'0': 248, '1': 20, '2': 611, '3': 5437, '4': 132, '5': 484, '6': 0, '7': 0}


# Replace Class Type

Drop -, cmu, and missing_wall classes

In [25]:
classes = {'0': '0', # Column Wall intersection
           '2': '1', # Floating Door
           '3': '2', # Illegible Text
           '5': '3'  # No stair direction
          } 

classes_to_omit = ['1','4']

In [26]:
def replace_class_n_with_class_name(file_path):
    '''for a txt file replace the class number with a class name'''
    new_lines = []
    infile = open(file_path, "r")
    lines = infile.readlines()

    if len(lines) == 0:
        pass
    else:
        for og_line in lines:
            class_n = og_line[0]
            if class_n in classes_to_omit: 
                continue
            class_str = classes[class_n]
            new_line = og_line.replace(class_n, class_str, 1)
            new_lines.append(new_line)

    with open(file_path, 'w') as outfile:
        outfile.write(''.join(new_lines))

In [29]:
!pwd

/workspace/yolo/yolo/yolov5/train


In [30]:
# replace class names for test and validation txt files

os.chdir("/workspace/yolo/yolo/yolov5/train")
for file in glob.glob("*.txt"):
    replace_class_n_with_class_name(file)
    
os.chdir("/workspace/yolo/yolo/yolov5/valid")
for file in glob.glob("*.txt"):
    replace_class_n_with_class_name(file)    
    

# Install W&B login to track model - only do once

In [31]:
!pip install wandb
!wandb login

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 1.7 MB 33.4 MB/s eta 0:00:01
     |████████████████████████████████| 144 kB 125.1 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 115.9 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 74.3 MB/s  eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=de1368f42d442ae6222687507aa325e2634c0a0a35d80ffafbb386fe494849e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-z_ak8gz1/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=72eb0b675b3528b5dad042467f6c1ffdfa6224e42c406641208832356ee55546
  Stored in directory: /tmp/pip-ephem-wheel-cache-z_ak8gz1/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=5a

In [37]:
!wandb login

wandb: Currently logged in as: auto-arki (use `wandb login --relogin` to force relogin)


In [39]:
import wandb

wandb.init(project="custom_yolov5", entity="dfristau")

# Learning

use smallest YOLO model to start yolov5s.pt

In [40]:
import torch
import torchvision

In [42]:
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 292 kB 34.2 MB/s eta 0:00:01


In [52]:
# reduce epochs and set cos-lr to false
os.chdir("/workspace/yolo/yolo/yolov5")
!python train.py\
    --data custom_dataset.yaml \
    --epochs 300 \
    --project custom_yolov5 \
    --bbox_interval 1 \
    --save-period 10 \
    #--img 1280 \
    --patience 10 \
    --weights yolov5x.pt \
    --optimizer 'AdamW' \
    --cos-lr false \
    --batch-size 10


requirements: /workspace/yolo/yolo/yolov5/Pillow>=8.4.0 not found, check failed.
wandb: Currently logged in as: auto-arki (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=, data=custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=custom_yolov5, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=10, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=1, artifact_alias=latest
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.1-39-g4effd06 torch 1.11.0a0+bfe5ad2 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epoc

     0/299     3.73G    0.1038   0.03095    0.0315        81       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.048      0.168     0.0159     0.0038

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     1/299     4.63G   0.07616   0.03053   0.01589        51       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.382      0.269      0.115     0.0387

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     2/299     4.63G   0.06848   0.02962   0.01203        37       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.265      0.239      0.177     0.0586

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     3/299     4.63G   0.05998    0.0

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892       0.51      0.633      0.548       0.35

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    26/299     4.63G   0.03548   0.02218  0.002648        68       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.613       0.66      0.639      0.413

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    27/299     4.63G   0.03503   0.02263  0.002724        49       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.581      0.668       0.62      0.401

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    28/299     4.63G   0.03499   0.02229  0.003003        53       640: 100%|███
               Class     Images     L

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.625      0.748      0.695      0.476

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    51/299     4.63G   0.03178   0.02142  0.001937        26       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.704       0.68      0.713      0.494

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    52/299     4.63G   0.03178   0.02046   0.00247        45       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892       0.72      0.694      0.727      0.505

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    53/299     4.63G   0.03177    0.0211  0.002248        48       640: 100%|███
               Class     Images     L


     Epoch   gpu_mem       box       obj       cls    labels  img_size
    76/299     4.63G   0.02934   0.01965  0.001651        45       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.602      0.604      0.634      0.439

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    77/299     4.63G   0.02878   0.01911  0.001906        29       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.685      0.705      0.709      0.497

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    78/299     4.63G   0.02976    0.0196  0.001779        50       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.645      0.697      0.683       0.47

     Epoch   gpu_mem       box      

...


     Epoch   gpu_mem       box       obj       cls    labels  img_size
   177/299     4.63G   0.02471   0.01701 0.0008197        66       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.758       0.71      0.741      0.535

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   178/299     4.63G   0.02542   0.01638  0.001076        43       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.722      0.711      0.728      0.523

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   179/299     4.63G   0.02424   0.01631  0.001021        42       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.755      0.668      0.726      0.523
Saving model artifact on epoch 180

 

   202/299     4.63G   0.02357   0.01591 0.0007949        37       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.686      0.745      0.731      0.529

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   203/299     4.63G   0.02352   0.01589 0.0006915        78       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.744      0.713      0.738      0.528

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   204/299     4.63G    0.0239   0.01579  0.001119        31       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892       0.76      0.675      0.721      0.517

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   205/299     4.63G   0.02378   0.01

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.716      0.725      0.721      0.522

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   228/299     4.63G   0.02237   0.01503 0.0009107        46       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.746      0.714      0.731      0.529

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   229/299     4.63G   0.02338   0.01546 0.0008338        60       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.745      0.719      0.732      0.529
Saving model artifact on epoch 230

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   230/299     4.63G   0.02227    0.0146  0.000651        66       640: 100%|███
  

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.725      0.714       0.72      0.523

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   253/299     4.63G   0.02171   0.01468 0.0006245        41       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.725      0.708      0.719      0.523

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   254/299     4.63G   0.02164   0.01498 0.0006532        53       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.747      0.687      0.718      0.522

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   255/299     4.63G   0.02144   0.01407 0.0005749        75       640: 100%|███
               Class     Images     L

requirements: /workspace/yolo/yolo/yolov5/Pillow>=8.4.0 not found, check failed.
Exception in thread Thread-23:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/workspace/yolo/yolo/yolov5/utils/plots.py", line 218, in plot_images
    annotator.text((x + 5, y + 5 + h), text=Path(paths[i]).name[:40], txt_color=(220, 220, 220))  # filenames
  File "/workspace/yolo/yolo/yolov5/utils/plots.py", line 116, in text
    w, h = self.font.getsize(text)  # text width, height
AttributeError: 'NoneType' object has no attribute 'getsize'
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        514       1892      0.742      0.717      0.751      0.541
Column_Wall_Intersection        514        118      0.654      0.449      0.503      0.381
       floati

'0' # Column Wall intersection <br>
'1', # Floating Door <br>
'2', # Illegible Text <br>
'3'  # No stair direction <br>
